<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Google Calendar - Get List of Events

This notebook will help us out in creating, or getting the list of events

**Tags**: #naas #google #calendar

**Author**: [Sanjeet Attili](https://www.linkedin.com/in/sanjeet-attili-760bab190)

## Input

**Steps to follow for setting up Google Calendar API credentials:**
1. First Visit the [google cloud console](https://console.cloud.google.com/apis/dashboard) and login with your google account

2. Create a new project and then on the dashboard at the top click on "enable apis and services" button.

3. Browse for Google calendar API and then once it is found, click on it and you will find "enable" button within that.

4. Once you have enabled it, you will be shown a new overview page of the Google Calendar API, over there you will have this message written "To use this API, you may need credentials. Click 'Create credentials' to get started." and a button named "Create Credentials".

5. After clicking on that button, you will be shown some series of options first "which API are you using?", select Google calendar API in the dropdown menu, and then in our case, i have just selected user data to work on, but there is another option called "application data" as well, read the below given info and select according to your use case!

6. Moving on, we will be next getting the OAuth Consent screen section, fill in the app name, user support email and developer contact information options (since they are mandatory)

7. Once this is done, we will be seeing the scopes as an optional tab, and we will skip, and move onto Oauth client ID section. In my case i went with web application and provided default name as "web client 1".

8. Add url in "Authorised redirect URIs" section, "http://localhost:8080".

9. Once added save it and then move onto download the credentials json file. Once downloaded we now go into the Oauth consent screen section, check for test users option and add your google account for using this calendar api

10. Congrats! You are now done with the setup and will be shown an option to download the credentials json file. Download and save it in your desktop. We will be using it in this notebook.

**Install the Google Client Library**

In [3]:
# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

**Install datefinder library for easy date interpretrability for end users**

In [2]:
!pip install datefinder

### Imports

In [1]:
import datetime, datefinder
import os

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

### Variables

Refer this link for more details on various scopes: https://developers.google.com/calendar/api/guides/auth#OAuth2Authorizing

In [2]:
# If scopes are modified, then the token.josn files 
# created using the below scripts should also be deleted
SCOPES = ['https://www.googleapis.com/auth/calendar']
Email = "Your_email_here"

## Model

**In events.list() function**

"calendarId" is the calendar identifier and can either be the email address of the calendar on which to create the event or a special keyword 'primary' which will use the primary calendar of the logged in user.

If you don't know the email address of the calendar you would like to use, you can check it either in the calendar's settings
of the Google Calendar web UI (in the section "Calendar Address") or you can look for it in the result of the calendarList.list() call.

**For detailed information regarding events list parameters click [here](https://developers.google.com/calendar/api/v3/reference/events/list#parameters)**

In [3]:
# Here token path argument refers to the json file which will be saved after the very 
# first run of the get_credentials function (once the authorization via google Oauth is done)
# This would be used for the further iterations, need not authorize via google Oauth all the time

# Credentials path refers to the client secret json file which we get from the
# google cloud console app creation process done above

def Get_credentials(SCOPES, token_path, credentials_path):
    creds = None
    if os.path.exists(token_path):
        creds = Credentials.from_authorized_user_file(token_path, SCOPES)
        
    # If there are no (valid) credentials available, let the user log in.
    
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                credentials_path, SCOPES)
            creds = flow.run_local_server()
        
        # Save the credentials for the next run
        with open(token_path, 'w') as token:
            token.write(creds.to_json())
    
    return creds

**Note**:

Use datefinder package for easy input taking from user's perspective.

Have start and end times as arguments in the below function for filtering the list of events range that is to be included


In [4]:
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes 
# for the first time.

def Get_list_of_events(SCOPES, token_path, credentials_path, calendar_id = 'primary', start_date= None, end_date = None, max_results = 20):
    try:
        creds = Get_credentials(SCOPES, token_path, credentials_path)
        service = build('calendar', 'v3', credentials=creds)
        
        # now = datetime.datetime.utcnow().isoformat() + 'Z'  # 'Z' indicates UTC time
        # ISO format is year month day time(in hours minutes econds and milliseconds)
        
        print('Getting the upcoming 10 events')
        if start_date!=None and end_date!=None:
            
            start_date = list(datefinder.find_dates(start_date))[0]
            end_date = list(datefinder.find_dates(end_date))[0]
            
            events_result = service.events().list(calendarId = calendar_id, 
                                              timeMin=start_date.strftime("%Y-%m-%dT%H:%M:%S"),
                                              timeMax = end_date.strftime("%Y-%m-%dT%H:%M:%S"),
                                              maxResults=max_results, singleEvents=True,
                                              orderBy='startTime').execute()
        else:
            events_result = service.events().list(calendarId = calendar_id,
                                              maxResults=max_results, singleEvents=True,
                                              orderBy='startTime').execute()
        
        events = events_result.get('items', [])

        if not events:
            print('No upcoming events found.')
            return

        # Prints the start date and name of the next 10 events
        for event in events:
            start = event['start'].get('dateTime', event['start'].get('date'))
            print(start, event['summary'])

    except HttpError as error:
        print('An error occurred: %s' % error)
    
    return 'Done'

## Output

**Example for Datefinder library**

In [5]:
match = datefinder.find_dates("5 march 9pm")
list(match)

[datetime.datetime(2022, 3, 5, 21, 0)]

In [6]:
common_mail_domains = ['gmail.com','yahoo.com', 'outlook.com']

In [ ]:
# Arguments to be provided for the function:

# 1. SCOPES (defined above)

# 2. token_path -> The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes 
# for the first time.

# 3. credentials_path -> Credentials path refers to the client secret json file which we get from the
# google cloud console app creation process done above.

# 4. calendar_id (default = 'primary') you can provide your emailID here for getting your calender events.

# If you want to access the primary calendar of the currently logged in user, use the default "primary" keyword option

# 5. max_results -> maximum no. of events to be returned (default value is 20).
# According to google calendar API the page size can never be largger than 2500 events

# 6. start_date and end_date (both default values = None). This can be used to filter the events starting from start_date
# and ending with end_date arguments. Either both should be None or both with some dates.

# start and end date format:
# Acceptable format -> "5th may 2022 9PM" or "5 may 9pm" or "5th may" time would be default 00:00

Get_list_of_events(SCOPES, "token.json" ,"client_secret.json", calendar_id = Email,
                   start_date = "16th march", end_date = "17th march 11:59PM",
                   max_results = 10)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=265869097269-7n2o7ql0qi59oqvcuht6p6jjom2t7ck5.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=kDtrkJzUhCmCTAPg4tP3dK5EKQo6r5&access_type=offline
